In [2]:
import warnings

import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings("ignore")

In [3]:
def create_train_test(input_data, time_step=60):
    data = input_data.reshape(-1, 1)

    # Приводим данные к диапазону от 0 до 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Создаем тренировочный и тестовый наборы данных
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]

    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)

    # Изменяем форму данных для входа в LSTM
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    return X_train, y_train, X_test, y_test, scaler


def create_dataset(dataset, time_step=1):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)


def create_model(time_step=60):
    # Создаем модель LSTM
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_absolute_percentage_error')

    return model



In [4]:
df = pd.read_csv('data/portfolio_data.csv')
X_train, y_train, X_test, y_test, scaler = create_train_test(df['DPZ'].values, time_step=60)

model = create_model(time_step=60)
model.fit(X_train, y_train, batch_size=1, epochs=1)

predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

print(model.summary())

1155/1155 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 17.2665
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,705 (373.85 KB)

 Trainable params: 31,901 (124.61 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 63,804 (249.24 KB)

None


In [7]:
print(X_test[1])

[[0.68354655]
 [0.68466553]
 [0.67071886]
 [0.67803182]
 [0.68410604]
 [0.68338672]
 [0.71327802]
 [0.70300778]
 [0.6978528 ]
 [0.70109763]
 [0.70173858]
 [0.71363593]
 [0.70514349]
 [0.72084664]
 [0.70750701]
 [0.70005604]
 [0.70105742]
 [0.71960482]
 [0.72080656]
 [0.72998002]
 [0.72873821]
 [0.7161597 ]
 [0.72837765]
 [0.72028574]
 [0.73767136]
 [0.72032582]
 [0.72421154]
 [0.72128724]
 [0.71848307]
 [0.72040592]
 [0.71575919]
 [0.72389112]
 [0.74315946]
 [0.74728555]
 [0.75730025]
 [0.75024986]
 [0.75245305]
 [0.72176804]
 [0.72861802]
 [0.79703878]
 [0.78826577]
 [0.76146624]
 [0.78530139]
 [0.77168142]
 [0.77789057]
 [0.78754477]
 [0.79659805]
 [0.79623756]
 [0.7942346 ]
 [0.79347347]
 [0.7942346 ]
 [0.77913238]
 [0.78297802]
 [0.78393939]
 [0.78077478]
 [0.77400485]
 [0.78806548]
 [0.78742453]
 [0.79255213]
 [0.80356828]]


In [11]:
model.predict(X_test[1:2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


array([[0.7344914]], dtype=float32)

In [14]:
model.info()

AttributeError: 'Sequential' object has no attribute 'info'